In [1]:
import os, gc
from tqdm import tqdm

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate
from langchain.vectorstores import Chroma, FAISS
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFacePipeline
#from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA

import torch

import warnings
warnings.filterwarnings('ignore')

print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))

1
NVIDIA GeForce RTX 3080 Ti


In [2]:
loader = PyPDFLoader("pdf_samples/1716891429607.pdf")
pages = loader.load_and_split()

print(pages[11].page_content)

II БҮЛЭГ.ӨГӨГДЛИЙН ХҮСНЭГТ
ТШЗ-ны 
холбогдох 
заалтА.ЕРӨНХИЙ ЗҮЙЛ
ТШЗ 1.1Захиалагчийн нэр: Монгол шуудан ХК
ТШЗ 1.1Зарлалын дугаар: МШ/20240202070
Тендер шалгаруулалтын нэр: Принтерийн хор
Тендер шалгаруулалтын багцын тоо болон нэр: Байхгүй
ТШЗ 2.1Санхүүжилтийн эх үүсвэр: Өөрийн хөрөнгө
Нийт төсөвт өртөг: 43,000,000 /дөчин гурван сая / төгрөг
Б.АГУУЛГА
ТШЗ 9.2Тендерийн баримт бичгийн талаар тодруулга авах хаяг:
Тендерийн баримт бичгийн талаар тодруулга авах хүсэлтийг хүлээн авах  
хугацааны хязгаар тендер хүлээн авах эцсийн хугацаанаас ажлын 2-оос  
доошгүй  хоногийн  өмнө  байна.  Тодруулга,  нэмэлт  мэдээлэл  авах  
хүсэлтийн хариуг хүсэлт гаргагчийн нэрийг дурдалгүйгээр албан бичгээр  
гаргаж,  түүний  хуулбарыг  Төрийн  худалдан  авах  ажиллагааны  цахим  
систем  (www.tender.gov.mn)-д  нийтлэх  бөгөөд  тус  тодруулга,  нэмэлт  
мэдээллийг бүх тендерт оролцогчдод нэгэн зэрэг хүргүүлсэнд тооцно.  
Иймд тендерт оролцохыг сонирхогч этгээдүүд дээрх цахим систем дэх  
тендерийн  урилгын

In [3]:
docs = loader.load()

r_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 50)
docs_split = r_splitter.split_documents(docs)

In [4]:
embedding = HuggingFaceEmbeddings(
    #model_name = "tugstugi/bert-base-mongolian-cased",
    #model_name = "sentence-transformers/LaBSE",
    model_name = "bayartsogt/albert-mongolian",
    model_kwargs = {"device": "cuda"}, 
    encode_kwargs = {"normalize_embeddings": True}
)

No sentence-transformers model found with name bayartsogt/albert-mongolian. Creating a new one with mean pooling.


config.json:   0%|          | 0.00/680 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/47.3M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/108 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/950k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

# Chroma test

In [5]:
vector_store = Chroma.from_documents(documents = docs_split, 
                                     embedding = embedding,
                                     persist_directory = 'db/')

In [10]:
prompt = 'санхүүгийн чадавхын шаардлага хэд вэ'

docs = vector_store.similarity_search(prompt)

print(len(docs))
for idx, doc in enumerate(docs):
    print(f"Returned: {idx + 1}")
    print(doc.page_content, '\n')

4
Returned: 1
баталгаажуулсан байна.
6.Бараанд  тавигдах  6.1.Дотоодын үйлдвэрлэлээр хангах боломжтой чанар, 

Returned: 2
санаа өгөх зорилгоор оруулсан ба тухайн гэрээний онцлогийг харгалзан захиалагч  
эцэслэнэ. 

Returned: 3
тавьсан  шаардлагад  нийцсэн,  бодитой  бөгөөд  өрсөлдөх  чадвартай  тендер  
бэлтгэхэд оролцогчдод зайлшгүй чухал нөхцөл болно. 
Техникийн  тодорхойлолтыг  бэлтгэхэд  өрсөлдөөнийг  өрнүүлэх  зарчмыг  
баримтлах ёстой бөгөөд худалдан авах барааны шинж чанар, материал, үйл  
ажиллагааны  шаардагдах  стандартын  талаар  тодорхой  тусгана.  Эдгээрийг  
хангасан  нөхцөлд  худалдан  авах  ажиллагааны  хэмнэлттэй,  үр  ашигтай, 

Returned: 4
асуудлаар  Захиалагчтай  харилцах  эрхтэй  
Нийлүүлэгчээс томилогдсон этгээдийг;
1.1.14.“Гэнэтийн буюу давагдашгүй хүчний шинжтэй  
онцгой нөхцөл байдал“ гэж талуудын хараа хяналтаас  
гадуур  болсон,  урьдчилан  таамаглах  боломжгүй, 



# FAISS test

In [11]:
faiss_index = FAISS.from_documents(documents = docs_split, 
                                   embedding = embedding,
                                   persist_directory = 'db/')

ImportError: Could not import faiss python package. Please install it with `pip install faiss-gpu` (for CUDA supported GPU) or `pip install faiss-cpu` (depending on Python version).

In [12]:
!pip install faiss-gpu

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu


In [ ]:
pip install faiss-cpu